In [1]:
import sys
from collections import deque
import heapq


%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import lines

from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
import time

# Needed to hide warnings in the matplotlib sections
import warnings
warnings.filterwarnings("ignore")

In [2]:
class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

In [3]:
class Node:

    """A node in a search tree. Contains a pointer to the parent (the node
    that this is a successor of) and to the actual state for this node. Note
    that if a state is arrived at by two paths, then there are two nodes with
    the same state.  Also includes the action that got us to this state, and
    the total path_cost (also known as g) to reach the node.  Other functions
    may add an f and h value; see best_first_graph_search and astar_search for
    an explanation of how the f and h values are handled. You will not need to
    subclass this class."""

    def __init__(self, state, parent=None, action=None, path_cost=0):
        """Create a search tree Node, derived from a parent by an action."""
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def __repr__(self):
        return "<Node {}>".format(self.state)

    def __lt__(self, node):
        return self.state < node.state

    def expand(self, problem):
        """List the nodes reachable in one step from this node."""
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        """[Figure 3.10]"""
        next_state = problem.result(self.state, action)
        next_node = Node(next_state, self, action,
                    problem.path_cost(self.path_cost, self.state,
                                      action, next_state))
        return next_node
    
    def solution(self):
        """Return the sequence of actions to go from the root to this node."""
        return [node.action for node in self.path()[1:]]

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

    # We want for a queue of nodes in breadth_first_graph_search or
    # astar_search to have no duplicated states, so we treat nodes
    # with the same state as equal. [Problem: this may not be what you
    # want in other contexts.]

    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state

    def __hash__(self):
        return hash(self.state)

In [4]:
class injection:
    """Dependency injection of temporary values for global functions/classes/etc.
    E.g., `with injection(DataBase=MockDataBase): ...`"""

    def __init__(self, **kwds):
        self.new = kwds

    def __enter__(self):
        self.old = {v: globals()[v] for v in self.new}
        globals().update(self.new)

    def __exit__(self, type, value, traceback):
        globals().update(self.old)


def memoize(fn, slot=None, maxsize=32):
    """Memoize fn: make it remember the computed value for any argument list.
    If slot is specified, store result in that slot of first argument.
    If slot is false, use lru_cache for caching the values."""
    if slot:
        def memoized_fn(obj, *args):
            if hasattr(obj, slot):
                return getattr(obj, slot)
            else:
                val = fn(obj, *args)
                setattr(obj, slot, val)
                return val
    else:
        @functools.lru_cache(maxsize=maxsize)
        def memoized_fn(*args):
            return fn(*args)

    return memoized_fn


def name(obj):
    """Try to find some reasonable name for the object."""
    return (getattr(obj, 'name', 0) or getattr(obj, '__name__', 0) or
            getattr(getattr(obj, '__class__', 0), '__name__', 0) or
            str(obj))


def isnumber(x):
    """Is x a number?"""
    return hasattr(x, '__int__')


def issequence(x):
    """Is x a sequence?"""
    return isinstance(x, collections.abc.Sequence)


def print_table(table, header=None, sep='   ', numfmt='{}'):
    """Print a list of lists as a table, so that columns line up nicely.
    header, if specified, will be printed as the first row.
    numfmt is the format for all numbers; you might want e.g. '{:.2f}'.
    (If you want different formats in different columns,
    don't use print_table.) sep is the separator between columns."""
    justs = ['rjust' if isnumber(x) else 'ljust' for x in table[0]]

    if header:
        table.insert(0, header)

    table = [[numfmt.format(x) if isnumber(x) else x for x in row]
             for row in table]

    sizes = list(map(lambda seq: max(map(len, seq)), list(zip(*[map(str, row) for row in table]))))

    for row in table:
        print(sep.join(getattr(str(x), j)(size) for (j, size, x) in zip(justs, sizes, row)))


def open_data(name, mode='r'):
    aima_root = os.path.dirname(__file__)
    aima_file = os.path.join(aima_root, *['aima-data', name])

    return open(aima_file, mode=mode)


def failure_test(algorithm, tests):
    """Grades the given algorithm based on how many tests it passes.
    Most algorithms have arbitrary output on correct execution, which is difficult
    to check for correctness. On the other hand, a lot of algorithms output something
    particular on fail (for example, False, or None).
    tests is a list with each element in the form: (values, failure_output)."""
    return mean(int(algorithm(x) != y) for x, y in tests)


In [5]:
class PriorityQueue:
    """A Queue in which the minimum (or maximum) element (as determined by f and
    order) is returned first.
    If order is 'min', the item with minimum f(x) is
    returned first; if order is 'max', then it is the item with maximum f(x).
    Also supports dict-like lookup."""

    def __init__(self, order='min', f=lambda x: x):
        self.heap = []
        if order == 'min':
            self.f = f
        elif order == 'max':  # now item with max f(x)
            self.f = lambda x: -f(x)  # will be popped first
        else:
            raise ValueError("Order must be either 'min' or 'max'.")

    def append(self, item):
        """Insert item at its correct position."""
        heapq.heappush(self.heap, (self.f(item), item))

    def extend(self, items):
        """Insert each item in items at its correct position."""
        for item in items:
            self.append(item)

    def pop(self):
        """Pop and return the item (with min or max f(x) value)
        depending on the order."""
        if self.heap:
            return heapq.heappop(self.heap)[1]
        else:
            raise Exception('Trying to pop from empty PriorityQueue.')

    def __len__(self):
        """Return current capacity of PriorityQueue."""
        return len(self.heap)

    def __contains__(self, key):
        """Return True if the key is in PriorityQueue."""
        return any([item == key for _, item in self.heap])

    def __getitem__(self, key):
        """Returns the first value associated with key in PriorityQueue.
        Raises KeyError if key is not present."""
        for value, item in self.heap:
            if item == key:
                return value
        raise KeyError(str(key) + " is not in the priority queue")

    def __delitem__(self, key):
        """Delete the first occurrence of key."""
        try:
            del self.heap[[item == key for _, item in self.heap].index(True)]
        except ValueError:
            raise KeyError(str(key) + " is not in the priority queue")
        heapq.heapify(self.heap)

In [6]:
class EightPuzzle(Problem):
    """ The problem of sliding tiles numbered from 1 to 8 on a 3x3 board, where one of the
    squares is a blank. A state is represented as a tuple of length 9, where  element at
    index i represents the tile number  at index i (0 if it's an empty square) """

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        """ Define goal state and initialize a problem """
        super().__init__(initial, goal)

    def find_blank_square(self, state):
        """Return the index of the blank square in a given state"""

        return state.index(0)

    def actions(self, state):
        """ Return the actions that can be executed in the given state.
        The result would be a list, since there are only four possible actions
        in any given state of the environment """

        possible_actions = ['UP', 'DOWN', 'LEFT', 'RIGHT']
        index_blank_square = self.find_blank_square(state)

        if index_blank_square % 3 == 0:
            possible_actions.remove('LEFT')
        if index_blank_square < 3:
            possible_actions.remove('UP')
        if index_blank_square % 3 == 2:
            possible_actions.remove('RIGHT')
        if index_blank_square > 5:
            possible_actions.remove('DOWN')

        return possible_actions

    def result(self, state, action):
        """ Given state and action, return a new state that is the result of the action.
        Action is assumed to be a valid action in the state """

        # blank is the index of the blank square
        blank = self.find_blank_square(state)
        new_state = list(state)

        delta = {'UP': -3, 'DOWN': 3, 'LEFT': -1, 'RIGHT': 1}
        neighbor = blank + delta[action]
        new_state[blank], new_state[neighbor] = new_state[neighbor], new_state[blank]

        return tuple(new_state)

    def goal_test(self, state):
        """ Given a state, return True if state is a goal state or False, otherwise """

        return state == self.goal

    def check_solvability(self, state):
        """ Checks if the given state is solvable """

        inversion = 0
        for i in range(len(state)):
            for j in range(i + 1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j] != 0:
                    inversion += 1

        return inversion % 2 == 0

    def h(self, node):
        """ Return the heuristic value for a given state. Default heuristic function used is 
        h(n) = number of misplaced tiles """

        return sum(s != g for (s, g) in zip(node.state, self.goal))

In [7]:
# Heuristics for 8 Puzzle Problem
import math

def linear(node):
    goal = (1, 2, 3, 4, 5, 6, 7, 8, 0)
    return sum([1 if node.state[i] != goal[i] else 0 for i in range(8)])

def manhattan(node):
    state = node.state
    index_goal = {0:[2,2], 1:[0,0], 2:[0,1], 3:[0,2], 4:[1,0], 5:[1,1], 6:[1,2], 7:[2,0], 8:[2,1]}
    index_state = {}
    index = [[0,0], [0,1], [0,2], [1,0], [1,1], [1,2], [2,0], [2,1], [2,2]]
    x, y = 0, 0
    
    for i in range(len(state)):
        index_state[state[i]] = index[i]
    
    mhd = 0
    
    for i in range(8):
        for j in range(2):
            mhd = abs(index_goal[i][j] - index_state[i][j]) + mhd
    
    return mhd

def max_heuristic(node):
    score1 = manhattan(node)
    score2 = linear(node)
    return max(score1, score2)

In [8]:
def breadth_first_graph_search(problem):
    """[Figure 3.11]
    Note that this function can be implemented in a
    single line as below:
    return graph_search(problem, FIFOQueue())
    """
    node = Node(problem.initial)
    if problem.goal_test(node.state):
        return node
    frontier = deque([node])
    explored = set()
    while frontier:
        node = frontier.popleft()
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                if problem.goal_test(child.state):
                    return child
                frontier.append(child)
    return None

def best_first_graph_search(problem, f, display=False):
    """Search the nodes with the lowest f scores first.
    You specify the function f(node) that you want to minimize; for example,
    if f is a heuristic estimate to the goal, then we have greedy best
    first search; if f is node.depth then we have breadth-first search.
    There is a subtlety: the line "f = memoize(f, 'f')" means that the f
    values will be cached on the nodes as they are computed. So after doing
    a best first search you can examine the f values of the path returned."""
    f = memoize(f, 'f')
    node = Node(problem.initial)
    frontier = PriorityQueue('min', f)
    frontier.append(node)
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            if display:
                print(len(explored), "paths have been expanded and", len(frontier), "paths remain in the frontier")
            return node
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.append(child)
            elif child in frontier:
                if f(child) < frontier[child]:
                    del frontier[child]
                    frontier.append(child)
    return None

def astar_search(problem, h=None, display=False):
    """A* search is best-first graph search with f(n) = g(n)+h(n).
    You need to specify the h function when you call astar_search, or
    else in your Problem subclass."""
    h = memoize(h or problem.h, 'h')
    return best_first_graph_search(problem, lambda n: n.path_cost + h(n), display) 

def astar_search_2(problem, h=manhattan, display=False):
    """A* search is best-first graph search with f(n) = g(n)+h(n).
    You need to specify the h function when you call astar_search, or
    else in your Problem subclass."""
    h = memoize(h or problem.h, 'h')
    return best_first_graph_search(problem, lambda n: n.path_cost + h(n), display)

def astar_search_3(problem, h=max_heuristic, display=False):
    """A* search is best-first graph search with f(n) = g(n)+h(n).
    You need to specify the h function when you call astar_search, or
    else in your Problem subclass."""
    h = memoize(h or problem.h, 'h')
    solution, explored, frontier = best_first_graph_search(problem, lambda n: n.path_cost + h(n), display) 
    return solution, explored, frontier


In [9]:
def puzzle_8_solver(file_path, algorithm):
    
    # read files in
    with open(file_path, 'r') as file:
        puzzle_s1_raw = file.read().split()
    puzzle_s1 = tuple(int(x if x != '_' else '0') for x in puzzle_s1_raw)
    
    # define function to get the required output
    def func_output(algo, problem, heuristic=None):
        
        # start timing
        start_time = time.perf_counter()
        
        # checking if heuristic argument was given
        if heuristic:
            solution = algo(problem, heuristic)
        
        # uniformed algo
        else:
            solution = algo(problem)
        
        # required output
        seq_actions = solution.solution()
        path = solution.path()
        tot_nodes = len(path)
        
        # edn timing
        end_time = time.perf_counter()
        
        # total time taken
        time_taken = end_time - start_time
        
        return tot_nodes, seq_actions, time_taken, path
    
    # fit puzzle in
    puzzle = EightPuzzle(puzzle_s1)
    
    # check for solvability
    is_solvable = puzzle.check_solvability(puzzle_s1)
    
    if is_solvable == False:
        print("Problem is not solvable.")
        return None 
    
    # dictionary to map algorithm names to their corresponding functions
    algo_dict = {
        'BFS': breadth_first_graph_search,
        'IDS': iterative_deepening_search,
        'h1': astar_search,
        'h2': astar_search_2,
        'h3': astar_search_3
    }
    
    if algorithm in algo_dict:
        return func_output(algo_dict[algorithm], puzzle)
    else:
        print(f"Algorithm {algorithm} is not recognized. The available algorithms are: BFS, IDS, h1, h2, h3")
        return None


In [10]:
file_path = '../Part2/S1.txt'
algorithm = 'h1'

# read files in
with open(file_path, 'r') as file:
    puzzle_s1_raw = file.read().split()
    puzzle_s1 = tuple(int(x if x != '_' else '0') for x in puzzle_s1_raw)


puzzle = EightPuzzle(puzzle_s1)
puzzle

In [11]:
puzzle_s1

(3, 5, 2, 6, 1, 7, 0, 8, 4)

In [12]:
puzzle_s1.index(0) % 3 == 0

True

In [13]:
puzzle = EightPuzzle(puzzle_s1)
puzzle.check_solvability(puzzle_s1)

True

In [14]:
puzzle.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [19]:
puzzle.initial

(3, 5, 2, 6, 1, 7, 0, 8, 4)

In [25]:
puzzle.initial

(3, 5, 2, 6, 1, 7, 0, 8, 4)

In [105]:

def best_first_graph_search(problem, f, display=False):
    f = memoize(f, 'f')
    node = Node(problem.initial)
    frontier = PriorityQueue('min', f)
    frontier.append(node)
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            if display:
                print(len(explored), "paths have been expanded and", len(frontier), "paths remain in the frontier")
            return node, explored
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.append(child)
            elif child in frontier:
                if f(child) < frontier[child]:
                    del frontier[child]
                    frontier.append(child)
    return None


In [106]:
def astar_search_2(problem, h=manhattan, display=False):
    """A* search is best-first graph search with f(n) = g(n)+h(n).
    You need to specify the h function when you call astar_search, or
    else in your Problem subclass."""
    h = memoize(h or problem.h, 'h')
    
    solution, explored = best_first_graph_search(problem, lambda n: n.path_cost + h(n), display)

    return solution, explored

In [107]:
solution, visited_nodes = astar_search_2(puzzle, manhattan, display=True)
path = solution.path()
path

2025 paths have been expanded and 1163 paths remain in the frontier


[<Node (3, 5, 2, 6, 1, 7, 0, 8, 4)>,
 <Node (3, 5, 2, 0, 1, 7, 6, 8, 4)>,
 <Node (0, 5, 2, 3, 1, 7, 6, 8, 4)>,
 <Node (5, 0, 2, 3, 1, 7, 6, 8, 4)>,
 <Node (5, 1, 2, 3, 0, 7, 6, 8, 4)>,
 <Node (5, 1, 2, 3, 8, 7, 6, 0, 4)>,
 <Node (5, 1, 2, 3, 8, 7, 6, 4, 0)>,
 <Node (5, 1, 2, 3, 8, 0, 6, 4, 7)>,
 <Node (5, 1, 2, 3, 0, 8, 6, 4, 7)>,
 <Node (5, 1, 2, 0, 3, 8, 6, 4, 7)>,
 <Node (5, 1, 2, 6, 3, 8, 0, 4, 7)>,
 <Node (5, 1, 2, 6, 3, 8, 4, 0, 7)>,
 <Node (5, 1, 2, 6, 3, 8, 4, 7, 0)>,
 <Node (5, 1, 2, 6, 3, 0, 4, 7, 8)>,
 <Node (5, 1, 2, 6, 0, 3, 4, 7, 8)>,
 <Node (5, 1, 2, 0, 6, 3, 4, 7, 8)>,
 <Node (0, 1, 2, 5, 6, 3, 4, 7, 8)>,
 <Node (1, 0, 2, 5, 6, 3, 4, 7, 8)>,
 <Node (1, 2, 0, 5, 6, 3, 4, 7, 8)>,
 <Node (1, 2, 3, 5, 6, 0, 4, 7, 8)>,
 <Node (1, 2, 3, 5, 0, 6, 4, 7, 8)>,
 <Node (1, 2, 3, 0, 5, 6, 4, 7, 8)>,
 <Node (1, 2, 3, 4, 5, 6, 0, 7, 8)>,
 <Node (1, 2, 3, 4, 5, 6, 7, 0, 8)>,
 <Node (1, 2, 3, 4, 5, 6, 7, 8, 0)>]

In [108]:
len(visited_nodes)

2025

In [119]:
list(visited_nodes)[-1]

(1, 5, 2, 0, 6, 3, 8, 7, 4)

In [29]:
solution.expand(puzzle)

[<Node (1, 2, 3, 4, 5, 0, 7, 8, 6)>, <Node (1, 2, 3, 4, 5, 6, 7, 0, 8)>]

In [17]:
for i in range(len(solution.path())):
    print(solution.path()[i])

<Node (3, 5, 2, 6, 1, 7, 0, 8, 4)>
<Node (3, 5, 2, 0, 1, 7, 6, 8, 4)>
<Node (0, 5, 2, 3, 1, 7, 6, 8, 4)>
<Node (5, 0, 2, 3, 1, 7, 6, 8, 4)>
<Node (5, 1, 2, 3, 0, 7, 6, 8, 4)>
<Node (5, 1, 2, 3, 8, 7, 6, 0, 4)>
<Node (5, 1, 2, 3, 8, 7, 6, 4, 0)>
<Node (5, 1, 2, 3, 8, 0, 6, 4, 7)>
<Node (5, 1, 2, 3, 0, 8, 6, 4, 7)>
<Node (5, 1, 2, 0, 3, 8, 6, 4, 7)>
<Node (5, 1, 2, 6, 3, 8, 0, 4, 7)>
<Node (5, 1, 2, 6, 3, 8, 4, 0, 7)>
<Node (5, 1, 2, 6, 3, 8, 4, 7, 0)>
<Node (5, 1, 2, 6, 3, 0, 4, 7, 8)>
<Node (5, 1, 2, 6, 0, 3, 4, 7, 8)>
<Node (5, 1, 2, 0, 6, 3, 4, 7, 8)>
<Node (0, 1, 2, 5, 6, 3, 4, 7, 8)>
<Node (1, 0, 2, 5, 6, 3, 4, 7, 8)>
<Node (1, 2, 0, 5, 6, 3, 4, 7, 8)>
<Node (1, 2, 3, 5, 6, 0, 4, 7, 8)>
<Node (1, 2, 3, 5, 0, 6, 4, 7, 8)>
<Node (1, 2, 3, 0, 5, 6, 4, 7, 8)>
<Node (1, 2, 3, 4, 5, 6, 0, 7, 8)>
<Node (1, 2, 3, 4, 5, 6, 7, 0, 8)>
<Node (1, 2, 3, 4, 5, 6, 7, 8, 0)>
